In [1]:
from dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from Classifier_module import Three_Layer_Classifier
from torch.autograd import Variable


/home/zhou907/.conda/envs/cent7/5.1.0-py36/GANS2/lib/python3.6/site-packages/uproot/reading.py:186: FutureWarning: XRootD 5.1.1 is not fully supported; either upgrade to 5.2.0+ or set

    open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource

  warnings.warn(message, FutureWarning)


In [2]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################
path_to_model = "/depot/cms/top/zhou907/threeLayerModel_mumu_1.pt"

event_type = "mumu"  # could be ee, emu, mumu
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

In [3]:
# model = 0
model = Three_Layer_Classifier()
model.load_state_dict(torch.load(path_to_model))
model.eval()

Three_Layer_Classifier(
  (model): Sequential(
    (0): Linear(in_features=80, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=3, bias=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
  )
)

In [4]:



data_object = ProductionModeDataset(file)
x = data_object.get_eval_data()


['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 'lbar_eta', 'l_phi', 'lbar_phi', '

In [5]:
weight = x[:,81]
target = x[:,80]
target = Variable(torch.from_numpy(target).type(torch.LongTensor))
y = np.transpose(x)
y = np.delete(y, [80, 81, 82], 0)
y = np.transpose(y)
input = Variable(torch.from_numpy(y).type(torch.Tensor))

In [6]:
model(input)[0,:]

tensor([32548.6191, 32148.6758, 33918.6953], grad_fn=<SliceBackward>)

In [7]:
target[0]

tensor(1)

In [8]:
from skorch import NeuralNetClassifier
from sklearn.inspection import permutation_importance

In [9]:
nn = NeuralNetClassifier(model, max_epochs=0, lr=0.00, batch_size=5000)
nn.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Three_Layer_Classifier(
    (model): Sequential(
      (0): Linear(in_features=80, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
      (4): Linear(in_features=256, out_features=3, bias=True)
      (5): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  ),
)

In [10]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [11]:
r = permutation_importance(nn, input, target, \
                           n_repeats=30, \
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{input_variables[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")


ttbar_phi0.155 +/- 0.005
nunubar_phi0.153 +/- 0.004
llbar_delta_phi0.150 +/- 0.004
nunubar_delta_phi0.149 +/- 0.004
llbar_phi0.148 +/- 0.004
bbbar_phi0.143 +/- 0.003
bbar_phi0.143 +/- 0.004
bbbar_delta_phi0.141 +/- 0.004
b_phi   0.141 +/- 0.004
wminus_phi0.140 +/- 0.004
top_phi 0.140 +/- 0.004
nubar_phi0.139 +/- 0.003
l_phi   0.138 +/- 0.004
bbarnubar_delta_phi0.136 +/- 0.003
lnubar_delta_phi0.134 +/- 0.004
bbar_eta0.132 +/- 0.004
nu_phi  0.131 +/- 0.004
tbar_phi0.130 +/- 0.004
lbar_phi0.130 +/- 0.004
bnubar_delta_phi0.130 +/- 0.004
wplus_phi0.129 +/- 0.004
b_eta   0.128 +/- 0.003
lbarnu_delta_phi0.127 +/- 0.004
wminusbbar_delta_phi0.127 +/- 0.003
lbarb_delta_phi0.126 +/- 0.003
lbbar_delta_phi0.124 +/- 0.003
lbar_eta0.123 +/- 0.003
l_eta   0.122 +/- 0.003
ttbar_pt0.121 +/- 0.003
wminusb_delta_phi0.119 +/- 0.004
bnu_delta_phi0.117 +/- 0.003
lbarb_delta_eta0.115 +/- 0.003
lb_delta_eta0.114 +/- 0.003
bbarnu_delta_phi0.111 +/- 0.003
ttbar_eta0.110 +/- 0.004
lbbar_delta_eta0.110 +/- 0.003
l

In [12]:
nn.score(input, target)

0.7782558806655192